In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake



import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np

from firedrake.petsc import PETSc

In [ ]:
# Get current path: all relative paths that you may use for input/output start from here.
#
# - Colab:  The default path is /content, and your GDrive folder is accessible (if mounted) at /content/drive/MyDrive
#
# - local:  If running on your local machine, current_path is the position WHERE YOU LAUNCHED THE NOTEBOOK KERNEL BY 'jupyter notebook'
#           and NOT the directory in which this ipynb file is saved.
#           If you want to modify your current path, go to the terminal, stop the kernel (ctrl-C + press y), then move to the desired path by
#           cd my/desired/path/starting/from/here
#           and then launch 'jupyter notebook'.
import os
current_path = os.getcwd()
print(current_path)

# my_io_path = '/content/drive/MyDrive/Colab Notebooks/CFD2324/'
my_io_path = current_path+"/"
print(my_io_path)

---
---
# Exercise 1

\begin{equation*}
\begin{cases}
(\boldsymbol{u}\cdot\nabla)\boldsymbol{u} - \frac{1}{\rm Re}\Delta \boldsymbol{u} + \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega, \\
{\rm div}\,\boldsymbol{u} = 0 & {\rm in} \ \Omega, \\
\boldsymbol{u} = \boldsymbol{i} & {\rm on} \ \Gamma_{\rm up}, \\
\boldsymbol{u} = \boldsymbol{0} & {\rm on}\ \Gamma_{\rm wall}=\partial\Omega\setminus\Gamma_{\rm up}, \\
\end{cases}
\end{equation*}

In [ ]:
# Build the mesh
n = 30
mesh = UnitSquareMesh(n, n, diagonal='crossed')

fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend()

In [ ]:
# Function spaces (mixed formulation)
V = VectorFunctionSpace(mesh, 'P', 1)
Q = FunctionSpace(mesh, 'P', 1)
W = MixedFunctionSpace([V, Q])

nullsp=MixedVectorSpaceBasis(W,[W.sub(0),VectorSpaceBasis(constant=True)])

# Data and boundary conditions
Re = 30.0
f = Constant((0.,0.))
x = SpatialCoordinate(mesh)


bc_wall= DirichletBC(W.sub(0),Constant((0.,0.)), (1,2,3))
bc_in=DirichletBC(W.sub(0),Constant((1.0,0.0)),(4))
#double parent

bcs = (bc_in,bc_wall)



### Variational problems

In [ ]:
u, p = TrialFunctions(W)
v, q = TestFunctions(W)

def nonlinear_iteration_forms(u, v, p, q, Re, f, u_old):
    # Newton iteration
    a_Newton = (1.0/Re)*inner(grad(u),grad(v))*dx   +   dot(dot(grad(u),u_old),v)*dx  +dot(dot(grad(u_old),u),v)*dx  -p*div(v)*dx +q*div(u)*dx  +0.5 *div(u)*inner(u_old,v)*dx +0.5 *div(u_old)*inner(u,v)*dx

    L_Newton = +inner(f,v)*dx +dot(dot(grad(u_old),u_old),v)*dx +0.5 *div(u_old)*inner(u_old,v)*dx

    return a_Newton, L_Newton


def supg_forms(u, v, p, q, Re, f, u_old, mesh,delta):

    ubar=Function(Functionspace(mesh,'DG',0))


    ubar.project(sqrt(inner(u_old,u_old)))
    h=CellDiameter(mesh)

    Re_k=h*ubar*Re
    one=Constant(1.0)

    delta_K=h/(ubar+1e-15) *conditional((gt(Re_k,one),one,Re_k))  #c++ ? operator


    delta_k=Function(FunctionSpace(u_old.function_space().mesh(),'DG',0)) #piece wise linear function



    lhs_supg=delta_k*dot(div(u),div(v))*dx
    rhs_supg=delta_k *dx







### Initialization and definition of the ***linear*** solver for each nonlinear iteration.

In [ ]:
# Initialization (wh = 0)
wh= Function(W)
uh, ph = wh.subfunctions

# vtk output for Paraview
basename = 'lab08_ex1_'
outfileU = File(my_io_path+"output/"+basename+"velocity.pvd")
outfileP = File(my_io_path+"output/"+basename+"pressure.pvd")
uh.rename("Velocity")   # this name will be used in Paraview
ph.rename("Pressure")   # this name will be used in Paraview
outfileU.write(uh)
outfileP.write(ph)

# Nonlinear solver
u_old = Function(V)
u_old.assign(uh)    # copy the dof's of uh (defined over W) into u_old (defined over V)
                    # Do not use 'u_old = uh', which would yield a "shallow copy", by which
                    # u_old would contain a sort of pointer to uh: in such case, any update of uh
                    # would immediately reflect on u_old, whilst we want to keep the two functions
                    # separate from one another, and update them only when actually intended.
p_old = Function(Q)
p_old.assign(ph)

a, L = nonlinear_iteration_forms(u, v, p, q, Re, f, u_old)

### Iterative algorithm for the solution of the nonlinear problem

In [ ]:
maxit = 100
it = 0
tol = 1e-4
err = tol+1     # >tol in order to enter the loop at the beginning

while it <= maxit and err > tol:

    it += 1

    param={'ksp_type':'preonly','ksp_pc_type' :'mumps'}

    pb=LinearVariationalProblem(a,L,wh,nullsp,bcs=bcs)

    solver=LinearVariationalSolver(pb,solver_parameters=param)

    solver.solve()

    uh, ph = wh.subfunctions

    err = (errornorm(uh, u_old, 'H1') / norm(u_old, 'H1') +
           errornorm(ph, p_old, 'L2') / norm(p_old, 'L2'))

    print("Iteration = ", it, " Error = ", err)
    uh.rename("Velocity")
    ph.rename("Pressure")
    outfileU.write(uh)
    outfileP.write(ph)

    u_old.assign(uh) # update the old solution
    p_old.assign(ph) # update the old solution

if it <= maxit
    print('Nonlinear solver converged in', it, 'iterations.')
else:
    print('Nonlinear solver did NOT converge!\nRelative error =', err, 'after', it, 'iterations.')

In [ ]:
fig, ax = plt.subplots()
col = tripcolor(ph, axes=ax)
plt.colorbar(col)
plt.title('pressure')
fig, ax = plt.subplots()
col = quiver(uh, axes=ax)
plt.colorbar(col)
plt.title('velocity')

In [ ]:
#ex3


In [ ]:
n = 30
mesh =RectangleMesh(3*n, n, 3, 1)

fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend()

In [ ]:
# Function spaces (mixed formulation)
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)


W = MixedFunctionSpace([V, Q])


u, p = TrialFunctions(W)
v, q = TestFunctions(W)


# Data and boundary conditions
Re = 30.0
f = Constant((0.,0.))
x = SpatialCoordinate(mesh)

u_in = (6*x[1]*(1-x[1]), 0.)


bc_top= DirichletBC(W.sub(0).sub(1),Constant(0.,0.), (4))

bc_bottom=DirichletBC(W.sub(0),Constant((0.0,0.0)),(3))

#bc_right=DirichletBC(W.sub(0),Constant((0.0,0.0)),(3))

bc_left=DirichletBC(W.sub(0),u_in,(1))

#double parent

bcs = (bc_left,bc_top,bc_bottom)



In [ ]:
    a = (1.0/Re)*inner(grad(u),grad(v))*dx   +   dot(dot(grad(u),u),v)*dx  +dot(dot(grad(u),u),v)*dx  -p*div(v)*dx +q*div(u)*dx

    L=0

In [ ]:
wh= Function(W)
uh, ph = wh.subfunctions

In [ ]:
  param={'ksp_type':'preonly','ksp_pc_type' :'mumps'}

  pb=NonlinearVariationalProblem(a,L,wh,bcs=bcs)

  solver=NonLinearVariationalSolver(pb,solver_parameters=param)

  solver.solve()

  uh, ph = wh.subfunctions

